In [1]:
import os
import random
import pickle
import numpy as np
import tensorflow as tf

os.environ['TF_DETERMINISTIC_OPS'] = '1'

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

def save_streams(name, data):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(data, f)

def load_streams(name):
    with open(f'{name}.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

2022-08-27 19:13:18.510632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-27 19:13:18.510690: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## ML

In [2]:
train_data = load_streams('TRAIN_ML_DATA')
test_data = load_streams('TEST_ML_DATA')

X_train, y_train = np.array(train_data['X']), np.array(train_data['y'])
y_train = np.array([d[0] for d in y_train])

X_test, y_test = np.array(test_data['X']), np.array(test_data['y'])
y_test = np.array([d[0] for d in y_test])

print(len(X_train), len(y_train)) # 150 150
print(X_train.shape, y_train.shape) # (150, 13, 100) (150,)

print(len(X_test), len(y_test)) # 40 40
print(X_test.shape, y_test.shape) # (40, 13, 100) (40,)

n_features = X_train.shape[1]
n_steps = X_train.shape[2]

150 150
(150, 13, 100) (150,)
40 40
(40, 13, 100) (40,)


In [3]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# print(dir(tf.keras.metrics))

In [4]:
# define model
model = Sequential()
model.add(
    LSTM(
        100,
        activation='sigmoid',
        input_shape=(n_features, n_steps),
        return_sequences=False
    )
)
model.add(Dense(1))
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#     loss=tf.keras.losses.BinaryCrossentropy(),
#     metrics=['binary_accuracy']
# )
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
# fit model
history = model.fit(X_train, y_train, epochs=360, batch_size=30)
# history = model.fit(X_train, y_train, epochs=1400, batch_size=40)
# history = model.fit(X_train, y_train, epochs=2000, batch_size=40, validation_split=0.1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/360


2022-08-27 19:13:23.424773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-08-27 19:13:23.427689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-27 19:13:23.427778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-27 19:13:23.427832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-27 19:13:23.427887: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

5/5 [==============================] - 1s 7ms/step - loss: 0.7091 - accuracy: 0.5400
Epoch 2/360
5/5 [==============================] - 0s 7ms/step - loss: 1.2278 - accuracy: 0.4800
Epoch 3/360
5/5 [==============================] - 0s 8ms/step - loss: 0.7592 - accuracy: 0.5533
Epoch 4/360
5/5 [==============================] - 0s 7ms/step - loss: 0.7680 - accuracy: 0.6000
Epoch 5/360
5/5 [==============================] - 0s 8ms/step - loss: 0.6763 - accuracy: 0.5800
Epoch 6/360
5/5 [==============================] - 0s 7ms/step - loss: 0.6558 - accuracy: 0.6133
Epoch 7/360
5/5 [==============================] - 0s 7ms/step - loss: 0.6476 - accuracy: 0.6467
Epoch 8/360
5/5 [==============================] - 0s 7ms/step - loss: 0.6536 - accuracy: 0.6667
Epoch 9/360
5/5 [==============================] - 0s 8ms/step - loss: 0.6255 - accuracy: 0.6733
Epoch 10/360
5/5 [==============================] - 0s 7ms/step - loss: 0.6512 - accuracy: 0.6000
Epoch 11/360
5/5 [=======================

In [5]:
# evaluate training
train_acc = model.evaluate(X_train, y_train, verbose=0)
print("Accuracy: %.2f%%" % (train_acc[1]*100))

Accuracy: 98.00%


In [6]:
# evaluate test
test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (test_acc[1]*100))

Accuracy: 62.50%


In [7]:
# from matplotlib import pyplot

# # plot loss during training
# pyplot.subplot(211)
# pyplot.title('Loss')
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# # plot accuracy during training
# pyplot.subplot(212)
# pyplot.title('Accuracy')
# pyplot.plot(history.history['accuracy'], label='train')
# pyplot.plot(history.history['val_accuracy'], label='test')
# pyplot.legend()
# pyplot.show()

In [7]:
### TEST

yhat = model.predict(X_test, verbose=0)
near_source_metrics = {'total': 0, 'correct': 0}
far_source_metrics = {'total': 0, 'correct': 0}
for y, yh in zip(y_test, yhat):
    near_source_metrics['total'] += y
    far_source_metrics['total'] += (not y)
    if yh < 0.5 and y == 0:
        far_source_metrics['correct'] += 1
    if yh >= 0.5 and y == 1:
        near_source_metrics['correct'] += 1
print(near_source_metrics, far_source_metrics)

                        # PREDICTED   # PREDICTED     # TOTAL      # ACCURACY          # TOTAL ACCURACY
                        # RIGHT       # WRONG
# ACTUAL NEAR SOURCE        6            1               7           6/7 = 86%
                                                                                            # 62.5%
# PREDICTED FAR SOURCE      19           14             33          19/33 = 58%

# TOTAL EVENTS                                          40

{'total': 7, 'correct': 5} {'total': 33, 'correct': 20}


In [8]:
### TRAIN


yhat = model.predict(X_train, verbose=0)
near_source_metrics = {'total': 0, 'correct': 0}
far_source_metrics = {'total': 0, 'correct': 0}
for y, yh in zip(y_train, yhat):
    near_source_metrics['total'] += y
    far_source_metrics['total'] += (not y)
    if yh < 0.5 and y == 0:
        far_source_metrics['correct'] += 1
    if yh >= 0.5 and y == 1:
        near_source_metrics['correct'] += 1
print(near_source_metrics, far_source_metrics)

                        # PREDICTED   # PREDICTED     # TOTAL      # ACCURACY          # TOTAL ACCURACY
                        # RIGHT       # WRONG
# ACTUAL NEAR SOURCE       76            3              79           76/79 = 96%
                                                                                            # 97.33%
# PREDICTED FAR SOURCE     70            1              71           70/71 = 99%

# TOTAL EVENTS                                          150

{'total': 79, 'correct': 77} {'total': 71, 'correct': 70}
